<a href="https://colab.research.google.com/github/sophia-szady/sj-project/blob/main/Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import numpy as np
import pandas as pd

In [60]:
data = np.array(pd.read_csv("data_set.csv"))

In [63]:
data.shape
X = data[:,:-1]
Y = data[:, -1:]
X_train = X[:4000,:]
X_test = X[4000:,:]
Y_train = Y[:4000,:]
Y_test = Y[4000:,:]

(4961, 133)